<a href="https://colab.research.google.com/github/HKH515/brownbear2polarbear/blob/master/brownbear2polarbear_cyclegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1966 (delta 0), reused 0 (delta 0), pack-reused 1961
Receiving objects: 100% (1966/1966), 7.90 MiB | 29.31 MiB/s, done.
Resolving deltas: 100% (1303/1303), done.


In [3]:
#!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install -r pytorch-CycleGAN-and-pix2pix/requirements.txt


    100% |████████████████████████████████| 591.8MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x608a6000 @  0x7fa20e7602a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 26.9MB/s 
    100% |████████████████████████████████| 1.4MB 3.9MB/s 
    100% |████████████████████████████████| 2.0MB 6.8MB/s 
    100% |████████████████████████████████| 204kB 5.1MB/s 
  Running setup.py bdist_wheel for visdom ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/ee/87/ce/a5023722374ca73b57fc8d4284ba6f973c01219b3c385a07e0
  Running setup.py bdist_wheel for torchfile ... - done
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom to

In [4]:
!mkdir brownbear2polarbear
#!mkdir brownbear2polarbear/trainA
#!mkdir brownbear2polarbear/trainB
!ls
%cd brownbear2polarbear
!ls
!ls "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/"
!unzip -o -qq "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/trainA.zip"
#%cd ..
#%cd trainB
!unzip -o -qq "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/trainB.zip"
%cd ..

brownbear2polarbear  gdrive  pytorch-CycleGAN-and-pix2pix  sample_data
/content/brownbear2polarbear
testA  testA.zip  testB  testB.zip  trainA  trainA.zip	trainB	trainB.zip
/content


In [0]:
# randomly shuffle the names of the files, as CycleGAN reads the files using os.walk in the default order, we need to shuffle to generalize the training set when using small dataset sizes
import os
import random
seq = list(range(len(os.listdir("/content/brownbear2polarbear/trainA"))))
random.shuffle(seq)
path = "/content/brownbear2polarbear/trainA"
for image in os.listdir("/content/brownbear2polarbear/trainA"):
  os.rename(os.path.join(path, image), os.path.join(path, "%s.jpg" % seq.pop(0)))
  
  
seq = list(range(len(os.listdir("/content/brownbear2polarbear/trainB"))))
random.shuffle(seq)
path = "/content/brownbear2polarbear/trainB"
for image in os.listdir("/content/brownbear2polarbear/trainB"):
  os.rename(os.path.join(path, image), os.path.join(path, "%s.jpg" % seq.pop(0)))



In [6]:
import os
import datetime
os.environ["output_directory"] = "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint_%s/" % datetime.datetime.now()
os.environ["directory_to_be_loaded"] = "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint/"
!echo saving checkpoint in $output_directory
!echo will copy checkpoint to $directory_to_be_loaded

saving checkpoint in gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint_2019-01-22 10:10:50.157734/
will copy checkpoint to gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint/


In [0]:
#!ln -s "gdrive/My Drive/Colab Notebooks/checkpoints" checkpoints
#data_path = "gdrive/My Drive/Colab Notebooks/brownbear2polarbear/"
checkpoint_path = "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint/opt.txt"
if os.path.exists(checkpoint_path):
  if not os.path.exists("/content/checkpoints"):
    !mkdir /content/checkpoints
  if not os.path.exists("/content/checkpoints/brownbear2polarbear_cyclegan"):
    !mkdir /content/checkpoints/brownbear2polarbear_cyclegan
  !cp "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint/." /content/checkpoints/brownbear2polarbear_cyclegan/ -r
  #!rm -rf /content/checkpoints/brownbear2polarbear_cyclegan/web/ -r
  !time python pytorch-CycleGAN-and-pix2pix/train.py --dataroot /content/brownbear2polarbear --name brownbear2polarbear_cyclegan --model cycle_gan --continue_train --max_dataset_size 20 --save_epoch_freq 50
else:
  !time python pytorch-CycleGAN-and-pix2pix/train.py --dataroot /content/brownbear2polarbear --name brownbear2polarbear_cyclegan --model cycle_gan --max_dataset_size 20 --save_epoch_freq 50
#!rm -rf "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_checkpoint/"

!mkdir "$output_directory"
!cp /content/checkpoints/brownbear2polarbear_cyclegan/* "$output_directory" -r
!rm -rf "$directory_to_be_loaded"
!cp -r "$output_directory" "$directory_to_be_loaded"


----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: /content/brownbear2polarbear  	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                      

In [0]:
!ls /content/checkpoints/brownbear2polarbear_cyclegan/web/images

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
path = "/content/checkpoints/brownbear2polarbear_cyclegan/web/images"
for image in os.listdir(path):
  plt.imshow(plt.imread(os.path.join(path, image)))
  plt.show()


In [0]:
%cd brownbear2polarbear
!ls
!ls "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/"
!unzip -o -qq "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/testA.zip"
!unzip -o -qq "/content/gdrive/My Drive/Colab Notebooks/brownbear2polarbear/testB.zip"
%cd ..

In [0]:
!time python pytorch-CycleGAN-and-pix2pix/test.py --dataroot /content/brownbear2polarbear --name brownbear2polarbear_cyclegan --model cycle_gan

In [0]:
!rm -rf "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_results/"
!mkdir "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_results/"
!cp -r /content/results/brownbear2polarbear_cyclegan/. "gdrive/My Drive/Colab Notebooks/brownbear2polarbear_results/"